In [280]:
import pandas as pd
import datetime as dt

1 - Загрузка данных

In [281]:
data = pd.read_csv('out.csv', encoding_errors='ignore')
data.head()

,author,title,time,arcticle_hubs,tags,total_comments,bookmarks,views,votes
0,Vin_Z,«Чтение — всему голова!» Переводим электронные...,2022-12-17 14:29:02,"['Python', 'Программирование', 'Софт ', 'Читал...","['python', 'epub', 'beautiful soup', 'софт']",0,0,49,Рейтинг
1,venanen,"Телепортируем мемы в telegram, или как вернуть...",2022-12-17 14:27:46,"['JavaScript', 'Node.JS', 'ВКонтакте API']","['Автопостинг', 'Telegram', 'Bot', 'vk api']",0,0,40,Рейтинг
2,alixplisov,gRPC — альтернатива REST API от Google,2022-12-17 14:06:37,"['Python', 'Разработка веб-сайтов']","['rest', 'grpc', 'python', 'postman', 'http', ...",1,4,348,Всего голосов 4: ↑2 и ↓2
3,AlexRussianPyth,Колдовской NeoVIM. Часть вторая. “Конфиг Всевл...,2022-12-17 13:36:23,"['Программирование', 'VIM', 'Разработка под Li...","['vim', 'nvim', 'конфиг', 'lsp']",0,7,383,Рейтинг
4,averkij,Сделал приложение для создания мультиязычных к...,2022-12-17 12:01:01,"['Open source', 'Программирование', 'Машинное ...","['natural language processing', 'параллельные ...",4,19,633,Всего голосов 16: ↑16 и ↓0


2 - Предобработка дынных

In [283]:
data.votes = data.votes.replace('Рейтинг', 'Всего голосов 0: ↑0 и ↓0')
total_votes = pd.DataFrame(data.votes.str.split(' ', expand=True))
total_votes.columns=['name_0', 'name_1', 'total_votes', 'positive_votes', 'and', 'negative_votes']
total_votes = total_votes.drop(['name_0', 'name_1', 'and'], axis=1)
total_votes.positive_votes = total_votes.positive_votes.str.lstrip('↑').astype(float)
total_votes.negative_votes = total_votes.negative_votes.str.lstrip('↓').astype(float)
total_votes.total_votes = total_votes.positive_votes + total_votes.negative_votes
total_votes.positive_votes = total_votes.positive_votes.astype(int)
total_votes.negative_votes = total_votes.negative_votes.astype(int)
total_votes.total_votes = total_votes.total_votes.astype(int)
data = pd.concat([data, total_votes], axis=1)
data = data.drop('votes', axis=1)

In [284]:
data.total_comments = data.total_comments.astype(int)
data.bookmarks = data.bookmarks.astype(int)
data.time = pd.to_datetime(data.time).dt.strftime('%Y%m%d').astype(int)

In [285]:
for i in range(len(data.views)):
    value = list(data.views.loc[i])
    if 'K' in value:
        value.remove('K')
        if '.' in value:
            value.remove('.')
            value.append('00')
        else:
            value.append('000')
    elif 'M' in value:
        value.remove('M')
        if '.' in value:
            value.remove('.')
            value.append('00000')
        else:
            value.append('000000')
    data.views.loc[i] = ''.join(value)
data.views = data.views.astype(int)

C:\Users\leoni\AppData\Local\Temp\ipykernel_1772\2961380446.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.views.loc[i] = ''.join(value)


In [287]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287448 entries, 0 to 287447
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   author          287448 non-null  object
 1   title           287448 non-null  object
 2   time            287448 non-null  int32 
 3   arcticle_hubs   287448 non-null  object
 4   tags            287448 non-null  object
 5   total_comments  287448 non-null  int32 
 6   bookmarks       287448 non-null  int32 
 7   views           287448 non-null  int32 
 8   total_votes     287448 non-null  int32 
 9   positive_votes  287448 non-null  int32 
 10  negative_votes  287448 non-null  int32 
dtypes: int32(7), object(4)
memory usage: 16.4+ MB


In [288]:
data.author = data.author.astype('category')
data.author = data.author.cat.codes

In [289]:
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer = HashingVectorizer(n_features=100)
arcticle_hubs = data.arcticle_hubs
tags = data.tags
vectorizer = vectorizer.transform(arcticle_hubs)
data.arcticle_hubs = vectorizer.toarray()

vectorizer = HashingVectorizer(n_features=100)
vectorizer = vectorizer.transform(tags)
data.tags = vectorizer.toarray()

In [300]:
data.head()

,author,title,time,arcticle_hubs,tags,total_comments,bookmarks,views,total_votes,positive_votes,negative_votes
0,17712,«Чтение — всему голова!» Переводим электронные...,20221217,0.0,0.0,0,0,49,0,0,0
1,42807,"Телепортируем мемы в telegram, или как вернуть...",20221217,0.0,0.0,0,0,40,0,0,0
2,20362,gRPC — альтернатива REST API от Google,20221217,0.0,0.0,1,4,348,4,2,2
3,1234,Колдовской NeoVIM. Часть вторая. “Конфиг Всевл...,20221217,0.0,0.0,0,7,383,0,0,0
4,21886,Сделал приложение для создания мультиязычных к...,20221217,0.0,0.0,4,19,633,16,16,0


In [290]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287448 entries, 0 to 287447
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   author          287448 non-null  int32  
 1   title           287448 non-null  object 
 2   time            287448 non-null  int32  
 3   arcticle_hubs   287448 non-null  float64
 4   tags            287448 non-null  float64
 5   total_comments  287448 non-null  int32  
 6   bookmarks       287448 non-null  int32  
 7   views           287448 non-null  int32  
 8   total_votes     287448 non-null  int32  
 9   positive_votes  287448 non-null  int32  
 10  negative_votes  287448 non-null  int32  
dtypes: float64(2), int32(8), object(1)
memory usage: 15.4+ MB


3 - Обучение модели МО

In [291]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [292]:
y = data.views

lab_enc = LabelEncoder()
y = lab_enc.fit_transform(y)
x = data.drop(['title', 'views'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, train_size = 0.9)

In [293]:
linear_regression_model = LogisticRegression(random_state=0).fit(x_train,y_train)
pred = linear_regression_model.predict(x_test)
print(pred)

MemoryError: Unable to allocate 3.00 GiB for an array with shape (258703, 1559) and data type float64

In [ ]:
y

array([  1,   0, 162, ..., 738, 586, 316], dtype=int64)

In [ ]:
print(accuracy_score(pred, y_test))

0.023969385980170465


In [294]:
dtm = DecisionTreeClassifier(random_state=0).fit(x_train, y_train)
pred = dtm.predict(x_test)
print(pred)

MemoryError: could not allocate 3269459968 bytes

In [ ]:
print(accuracy_score(pred, y_test))

In [298]:
knb = KNeighborsClassifier(n_neighbors=3).fit(x_train, y_train)
pred = knb.predict(x_test)
print(pred)

[835 920 834 ... 908 911 835]


In [299]:
print(accuracy_score(pred, y_test))

0.09573838928509305


In [ ]:
rfc = RandomForestClassifier().fit(x_train, y_train)
pred = rfc.predict(x_test)
print(pred)

MemoryError: could not allocate 3269459968 bytes

In [ ]:
print(accuracy_score(pred, y_test))